In [1]:
%matplotlib widget

from imports import * 
from Head_content import head_content
from Left_content import left_content
from Center_content import  center_content
from Right_content import right_content
from plot_geometry import plot_geometry
from plot_incident_rays import plot_incident_rays
from plot_reflected_rays import plot_reflected_rays

plt.ioff()

H = head_content()
H_design = H.design()

C = center_content()
C_design = C.design()

R = right_content()
R_design = R.design()

L= left_content(center_instance=C, header_instance=H)
L_parabola = L.design_parabola()
L_ringArray = L.design_ring_array()
L_cylindrical = L.design_cylindrical()

app = widgets.AppLayout(header=H_design,
                        left_sidebar=L_parabola,
                        center=C_design,
                        right_sidebar=R_design,
                        footer=widgets.Button())
def changeLayout(*arg):
    global app
    value = H.collector_type.value
    if value == 1:
        app.left_sidebar = L_parabola
    if value == 2:
        app.left_sidebar = L_ringArray
    if value == 3:
        app.left_sidebar = L_cylindrical
        
def updateSourceGeometry(*arg):
    global app, L_parabola, L_ringArray, L_cylindrical
    L_parabola = L.design_parabola()
    L_ringArray = L.design_ring_array()
    L_cylindrical = L.design_cylindrical()
    
    value = H.collector_type.value
    if value == 1:
        app.left_sidebar = L_parabola
    if value == 2:
        app.left_sidebar = L_ringArray
    if value == 3:
        app.left_sidebar = L_cylindrical

H.collector_type.observe(changeLayout, 'value')
H.source_geometry.observe(updateSourceGeometry, 'value')


In [2]:
phi_tab = []
z_tab=[], 
rx=0.0
ry=0.0
inters=spy.Function
inters_lambda=any
yl_min=0.0
yl_max=0.0
zl_min=0.0
zl_max=0.0
parabola_rings=[]
h_2D = []
focs = []
# Update the list of varables for calculations
variables = L.variables
# Optical calculation classes instanciations
parabolic_calc = pmc((variables['parabola_focus_x'], variables['parabola_focus_y'], variables['parabola_height']), 
                                   variables['pt_source_coordinate'], variables['parabola_rot_angle'])
ring_calc = opra(Rin_0=variables['ringArray_internal_ray'], A_target=variables['ringArray_area'], 
                              N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                              h_max=variables['ringArray_h_max'])
# Geometry ploter instance
geo_plot = plot_geometry(fx=variables['parabola_focus_x'], fy=variables['parabola_focus_y'], h=variables['parabola_height'],
                         pt_source_pos=variables['pt_source_coordinate'], khoi=variables['parabola_rot_angle'], Rin_0=variables['ringArray_internal_ray'],
                         A_target=variables['ringArray_area'], N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                         h_max=variables['ringArray_h_max'], n_theta=variables['n_theta'], n_phi=variables['n_phi'],
                         center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc)

def plot_collector_geometry(*arg):
    
    global phi_tab, z_tab, rx, ry, inters, inters_lambda, yl_min, yl_max, zl_min, zl_max, parabola_rings, h_2D, focs
    
    value = H.collector_type.value
    # Update the list of varables for calculations
    variables = L.variables
    # Optical calculation classes instanciations
    parabolic_calc.update_init((variables['parabola_focus_x'], variables['parabola_focus_y'], variables['parabola_height']),
                               variables['pt_source_coordinate'], variables['parabola_rot_angle'])
    ring_calc.update_init(Rin_0=variables['ringArray_internal_ray'], A_target=variables['ringArray_area'],
                          N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                          h_max=variables['ringArray_h_max'])
    # Geometry ploter instance init update
    geo_plot.update_init(fx=variables['parabola_focus_x'], fy=variables['parabola_focus_y'], h=variables['parabola_height'],
                         pt_source_pos=variables['pt_source_coordinate'], khoi=variables['parabola_rot_angle'], Rin_0=variables['ringArray_internal_ray'],
                         A_target=variables['ringArray_area'], N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                         h_max=variables['ringArray_h_max'], n_theta=variables['n_theta'], n_phi=variables['n_phi'],
                         center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc)
    if value == 1:
        phi_tab, z_tab, rx, ry, inters, inters_lambda, yl_min, yl_max, zl_min, zl_max = \
        geo_plot.plot_parabola()
    if value == 2:
        parabola_rings, h_2D, focs = geo_plot.plot_ringArray()
    else:
        L.result_label.value = 'error in plot_collector_geometry function'

L.plot_surf_bt.on_click(plot_collector_geometry)
    

In [3]:
# Update the list of varables for calculations
variables = L.variables

# Incident rays plotter instance
inc_ray_plot = plot_incident_rays(n_phi=variables['n_phi'], n_theta=variables['n_theta'], inc_rayClr_value=variables['inc_rayClr_value'], 
                                  N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                                  h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                                  center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc)


def plot_incident_rays(*arg):
    # Update the list of varables for calculations
    variables = L.variables
    value = H.collector_type.value
    
    # incident rays plotter init update
    inc_ray_plot.update_init(n_phi=variables['n_phi'], n_theta=variables['n_theta'], inc_rayClr_value=variables['inc_rayClr_value'],
                             N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                             h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                             center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc)
    if value == 1:
        inc_ray_plot.plot_incident_parabola()
    if value == 2:
        inc_ray_plot.plot_incident_ringArray

L.plot_inc_bt.on_click(plot_incident_rays)

In [ ]:
# Update the list of varables for calculations
variables = L.variables

# Reflected rays plotter instance
refl_ray_plot = plot_reflected_rays(n_phi=variables['n_phi'], n_theta=variables['n_theta'], refl_rayClr_value=variables['refl_rayClr_value'], 
                                  N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                                  h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                                  center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc)



def plot_reflected_rays(*arg):
    # Update the list of varables for calculations
    variables = L.variables
    value = H.collector_type.value
    
    # Reflected rays plotter init update
    refl_ray_plot.update_init(n_phi=variables['n_phi'], n_theta=variables['n_theta'], refl_rayClr_value=variables['refl_rayClr_value'], 
                              N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                              h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                              center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc)
    
    if value == 1:
        refl_ray_plot.plot_reflected_parabola()
    if value == 2:
        refl_ray_plot.plot_reflected_ringArray()
    
L.plot_refl_bt.on_click(plot_reflected_rays)

In [4]:
display(app)

AppLayout(children=(VBox(children=(HTML(value=" <h1 style='color:blue;margin-:100px'; text-align:center><u><b>…